In [1]:
from folium.plugins import HeatMap
from folium.plugins import BeautifyIcon
from branca.colormap import linear
from branca.element import Template, MacroElement
from selenium import webdriver
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.graph_objects as go
import pandas as pd
import duckdb
import folium
import os
import random
import geopandas as gpd

pd.set_option('display.max_columns', None)

##### 變化率GIF圖

In [11]:
conn = duckdb.connect(database='warehouse\data_sandbox.duckdb')
df = conn.execute("SELECT * FROM youbike").fetch_df()
conn.close()

df = df[['sno', 'sna', 'sarea','fetch_time','latitude', 'longitude','available_rent_bikes','total','temperature','description']].sort_values(['sno','fetch_time'], ascending=True)
df["fetch_time"] = pd.to_datetime(df["fetch_time"])

# 定義計算變化率的函數
def calculate_change_rate(group):
    # 計算 available_rent_bikes 的變化率
    group['change_rate'] = group['available_rent_bikes'].pct_change().fillna(0)
    
    # 計算 fetch_time 的時間差（以分鐘計算）
    group['time_diff'] = group['fetch_time'].diff().dt.total_seconds() / 60
    
    # 如果時間差超過 20 分鐘，變化率設為 0
    group.loc[group['time_diff'] > 12, 'change_rate'] = 0
    
    return group

df_pct = df.groupby('sno',as_index=False).apply(calculate_change_rate)
df_available = df_pct.query("~time_diff.isna()").query("time_diff <12").query("change_rate != inf")

def generate_html(df, threshold=0.1):
    if os.path.exists(rf"D:\\sandbox_git\\warehouse\\bike_html\\bike_change_rate_map_{df.fetch_time.unique()[0].strftime('%Y%m%d%H%M')}.html"):
        print("exists")
        pass
    else:
        # Create a base map
        m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=13, tiles='CartoDB positron')

        # Normalize change_rate to a range between 0 and 1 for color mapping
        min_change_rate = df['change_rate'].min()
        max_change_rate = df['change_rate'].max()
        df['norm_change_rate'] = (df['change_rate'] - min_change_rate) / (max_change_rate - min_change_rate)
        df['norm_change_rate'] = df['norm_change_rate'].fillna(0)
        
        # Create a custom colormap
        colormap = linear.RdYlBu_11.scale(min_change_rate, max_change_rate)  # Red for high, blue for low
        colormap = colormap.to_step(n=10)  # Discretize the color map into 10 steps
        
        # Add points to the map
        for i, row in df.iterrows():
            # 如果 change_rate 絕對值小於 threshold，顯示為灰色
            if abs(row['change_rate']) < threshold:
                color = 'grey'
                fill_opacity = 0.01
                radius=1
            else:
                color = colormap(row['change_rate'])
                fill_opacity = 0.4
                radius=6
            
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),
                radius=radius, 
                color=color,  
                fill=True,
                fill_opacity=fill_opacity,
                popup=f"Change Rate: {row['change_rate']}"
            ).add_to(m)

        # Add the colormap legend
        colormap.add_to(m)

        # 添加顯示 fetch_time 的文字
        timestamp = df.fetch_time.unique()[0].strftime('%Y-%m-%d %H:%M:%S')
        print(timestamp)
        
        # 自定義的 HTML 模板，將時間顯示在右下角
        template = """
        {% macro html(this, kwargs) %}
        <div style="position: fixed;
                    bottom: 750px; right: 50px; z-index: 9999;
                    background-color: black; padding: 5px;
                    font-size: 24px; color: white">
        Fetched at: """ + timestamp + """
        </div>
        {% endmacro %}
        """
        
        # 添加模板到地圖
        macro = MacroElement()
        macro._template = Template(template)
        m.get_root().add_child(macro)

        # Save the map
        m.save(f"D:\\sandbox_git\\warehouse\\bike_html\\bike_change_rate_map_{df.fetch_time.unique()[0].strftime('%Y%m%d%H%M')}.html")

for time in df_available.fetch_time.unique():
    df = df_available.query("fetch_time == @time")
    generate_html(df)

# Set up the Selenium driver (ensure you have the correct driver installed)
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

# Folder paths
html_folder = 'D:/sandbox_git/warehouse/bike_html'
output_folder = 'D:/sandbox_git/warehouse/bike_images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert each HTML file to an image
for file_name in os.listdir(html_folder):
    if file_name.endswith('.html'):
        file_path = os.path.join(html_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name.replace('.html', '.png'))

        driver.get(f'file:///{file_path}')
        driver.set_window_size(1080, 1080)  # Adjust the window size as needed
        driver.save_screenshot(output_image_path)

driver.quit()

# Folder where images are saved
image_folder = 'D:/sandbox_git/warehouse/bike_images'
output_gif = 'D:/sandbox_git/warehouse/bike_change_rate_map.gif'

# Get all the images
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')]
image_files.sort()  # Ensure correct order

# Open images and create a GIF
images = [Image.open(img_file) for img_file in image_files]
images[0].save(output_gif, save_all=True, append_images=images[1:], duration=90, loop=0)

C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pct = df.groupby('sno',as_index=False).apply(calculate_change_rate)


exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
2024-09-30 21:30:02


C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = (df['change_rate'] - min_change_rate) / (max_change_rate - min_change_rate)
C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = df['norm_change_rate'].fillna(0)
C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:35: SettingWithCopyWarning: 
A value is trying to be set on a

2024-09-30 21:40:02


C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = (df['change_rate'] - min_change_rate) / (max_change_rate - min_change_rate)
C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = df['norm_change_rate'].fillna(0)


...

2024-10-01 03:10:01


C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = (df['change_rate'] - min_change_rate) / (max_change_rate - min_change_rate)
C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = df['norm_change_rate'].fillna(0)


2024-10-01 03:20:01


C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = (df['change_rate'] - min_change_rate) / (max_change_rate - min_change_rate)
C:\Users\tonyf\AppData\Local\Temp\ipykernel_12596\3990349628.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['norm_change_rate'] = df['norm_change_rate'].fillna(0)


2024-10-01 15:40:02
exists
exists
exists
exists
exists
exists
exists
exists


##### 週末起訖站點

In [2]:
# 讀取 GeoJSON 檔案
gdf = gpd.read_file('warehouse\週末起訖站點統計_202307.geojson')

# 檢視前五列數據
gdf.head()

,on_stop_id,off_stop_id,on_stop,off_stop,sum_of_txn_times,district_origin,district_destination,width,geometry
0,U101001,U101001,捷運科技大樓站,捷運科技大樓站,139,大安區,大安區,2.415385,"LINESTRING (121.5436 25.02605, 121.5436 25.02605)"
1,U101001,U101002,捷運科技大樓站,復興南路二段273號前,47,大安區,大安區,1.471795,"LINESTRING (121.5436 25.02605, 121.54357 25.02..."
2,U101001,U101003,捷運科技大樓站,國北教大實小東側門,24,大安區,大安區,1.235897,"LINESTRING (121.5436 25.02605, 121.54124 25.02..."
3,U101001,U101004,捷運科技大樓站,和平公園東側,121,大安區,大安區,2.230769,"LINESTRING (121.5436 25.02605, 121.54282 25.02..."
4,U101001,U101005,捷運科技大樓站,辛亥復興路口西北側,185,大安區,大安區,2.887179,"LINESTRING (121.5436 25.02605, 121.54299 25.02..."


In [ ]:
gdf[["on_stop_id","on_stop"]].drop_duplicates(subset=["on_stop_id"])

In [ ]:
# 創建一個字典來存放每個 on_stop_id 的顏色
on_stop_colors = {}
unique_on_stops = gdf['on_stop_id'].unique()

# 生成隨機顏色對應於每個 on_stop_id
colors = list(mcolors.CSS4_COLORS.values())  # 使用 CSS4 定義的顏色
random.shuffle(colors)  # 隨機打亂顏色列表

for i, on_stop_id in enumerate(unique_on_stops):
    on_stop_colors[on_stop_id] = colors[i % len(colors)]  # 確保顏色不會超出列表範圍

# 設定地圖的初始中心點與縮放級別
center_lat = gdf['geometry'].centroid.y.mean()
center_lon = gdf['geometry'].centroid.x.mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='cartodb positron')

# 遍歷資料並繪製對應的 LINESTRING
for _, row in gdf.iterrows():
    if row['geometry'].geom_type == 'LineString':
        coords = list(row['geometry'].coords)  # 取得 LINESTRING 的經緯度
        locations = [(lat, lon) for lon, lat in coords]  # folium 需要 (緯度, 經度) 的順序
        
        # 根據 on_stop_id 選擇顏色
        line_color = on_stop_colors[row['on_stop_id']]
        
        # 使用 folium 繪製線段，並根據 'width' 設置線條的寬度
        folium.PolyLine(
            locations=locations,
            color=line_color,
            weight=row['width'],  # 使用對應的線寬
            opacity=0.8
        ).add_to(m)

# 保存或顯示地圖
m.save("transit_line_map.html")  # 保存為 HTML 文件
m  # 若在 Jupyter Notebook 或 IPython 中執行，這樣可以直接顯示地圖

##### 需求預測視覺化

In [ ]:
# 假設你的資料存放在 df 中
df = gpd.read_file('D:\sandbox_git\warehouse\未設站區域需求預測_202303.geojson')   

# 建立一個類別對應顏色的字典
category_color_map = {
    '低': 'blue',
    '極低': 'purple',
    '中低': 'green',
    '中': 'orange',
    '高': 'red'
}

# 初始化地圖
fig = go.Figure()

# 遍歷每個類別並繪製對應的 POLYGON
for category, color in category_color_map.items():
    # 過濾資料集只保留該類別的資料
    filtered_df = df[df['category'] == category]
    
    # 繪製每一個 POLYGON
    for _, row in filtered_df.iterrows():
        # 獲取 POLYGON 的邊界座標
        if row['geometry'].geom_type == 'Polygon':
            x, y = row['geometry'].exterior.xy
            # 將 array.array 轉換為 list
            lon = list(x)
            lat = list(y)
            fig.add_trace(go.Scattermapbox(
                fill="toself",
                lon=lon,
                lat=lat,
                mode='lines',
                line=dict(width=2, color=color),
                fillcolor=color,
                name=category
            ))
            
# 設定 Mapbox 底圖樣式和中心位置
fig.update_layout(
    mapbox_style="dark",  # 使用 Mapbox 的 dark-v11 底圖
    mapbox_accesstoken="pk.eyJ1Ijoic2hpYm55IiwiYSI6ImNrcWtjMDg0NjA0anQyb3RnZnl0cDJkYmYifQ.hqyJUg0ZRzAZbcJwkfs0bQ",  # 替換成你的 Mapbox access token
    mapbox_zoom=10,  # 設定縮放比例
    mapbox_center={"lat": 25.0330, "lon": 121.5654},  # 根據你的資料調整中心位置
    title="Polygon Map with Category Colors"
)

# 顯示地圖
fig.show()


##### 站點分群結果

In [ ]:
# 讀取 CSV 檔案
df = pd.read_csv("D:\sandbox_git\warehouse\互補站點1.csv").rename(columns={'longitude1（經度）':'longitude1', 'latitude1（緯度）':'latitude1'})\
    .merge(gdf[["on_stop_id","on_stop"]].drop_duplicates(subset=["on_stop_id"]), left_on="stop_id1", right_on="on_stop_id", how="left" ).fillna("")

# 創建一個字典來存放每個 stop_id1 的顏色
stop_id_colors = {}
unique_stops = df['stop_id1'].unique()

# 生成隨機顏色對應於每個 stop_id1
colors = list(mcolors.CSS4_COLORS.values())  # 使用 CSS4 定義的顏色
random.shuffle(colors)  # 隨機打亂顏色列表

for i, stop_id in enumerate(unique_stops):
    stop_id_colors[stop_id] = colors[i % len(colors)]  # 確保顏色不會超出列表範圍

# 初始化地圖，以資料的平均經緯度作為地圖中心
center_lat = df['latitude1'].mean()
center_lon = df['longitude1'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='cartodb dark_matter')

# 繪製每個站點之間的線條
for _, row in df.iterrows():
    # 取得線段的經緯度
    locations = [(row['latitude1'], row['longitude1']), (row['latitude2'], row['longitude2'])]
    
    # 根據 stop_id1 選擇顏色
    line_color = stop_id_colors[row['stop_id1']]
    
    # 繪製兩站之間的線段，並根據 times 設置線寬
    folium.PolyLine(
        locations,
        color=line_color,
        weight=max(row['times'] / 200, 1),  # 將 times 作為線寬，縮小至 1/20，最小寬度設為 1 避免過細
        opacity=0.5  # 增加透明度避免線條過度疊加
    ).add_to(m)

# 添加 stop_id1 標記
for on_stop, group in df.groupby('on_stop'):
    # 取第一個經緯度來標示 stop_id1 的位置
    lat = group.iloc[0]['latitude1']
    lon = group.iloc[0]['longitude1']
    
    # 添加帶有 stop_id1 的 marker
    folium.Marker(
        location=[lat, lon],
        popup=f"Stop Name: {on_stop}",
        icon=BeautifyIcon(
            icon_shape='marker',
            border_color='blue',
            background_color='darkblue'
        )
    ).add_to(m)

# 保存或顯示地圖
m.save("commuting_flow_cleaned.html")  # 保存為 HTML 文件
m  # 若在 Jupyter Notebook 或 IPython 中執行，這樣可以直接顯示地圖

##### 轉乘

In [36]:
df = pd.read_csv(r"D:\sandbox_git\warehouse\202312_轉乘YouBike2.0票證刷卡資料.csv")
df["借車時間"] = pd.to_datetime(df["借車時間"])
df["還車時間"] = pd.to_datetime(df["還車時間"])

In [ ]:
df.sort_values("租借時數",ascending=False)

##### 見車率

In [4]:
gpd.read_file("D:\sandbox_git\warehouse\見車率_202307.geojson").query("category == '低'")

,stop_id,stop_name,capacity,latitude,longitude,category,geometry
752,U108087,東湖一號公園,10,25.07567,121.61180,低,POINT (121.6118 25.07567)
783,U108121,內湖路一段47巷,16,25.08781,121.55872,低,POINT (121.55872 25.08781)
1258,U119059,臺大共同教室北側,20,25.01595,121.53791,低,POINT (121.53791 25.01595)
1259,U119060,臺大共同教室東南側,12,25.01566,121.53786,低,POINT (121.53786 25.01566)
1264,U119065,臺大二號館,10,25.01699,121.53574,低,POINT (121.53574 25.01699)
1278,U119080,臺大小小福西南側,10,25.01539,121.53686,低,POINT (121.53686 25.01539)
1280,U119082,臺大四號館東北側,10,25.01703,121.53781,低,POINT (121.53781 25.01703)
